---
title: "2023_Kronensicherung_Plesse_003_PTQ"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_PTQ
## ## PTQ: Daten der Elastometer (PicusTreeQinetic Data = PTQ)

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment) und installiere das Paket treeqinetic (PTQ) inklusive kj_core und kj_logger und weiteren requirements.

## Arbeitsumgebung vorbereiten


Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend das extra geschriebenen Pakete PTQ. Fehler beim Import dieses Pakets sind ggf. Bugs. Es nutzte eine gemeinsame CodeBasis in den Paketen kj_core (Core-Package) und kj_logger (individualisiertes Logging des Verarbeitungs-Prozesses). Diese sollte i. d. R. über die requirements mit installiert werden.

### IMPORT: Importieren von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [ ]:
import numpy as np
import pandas as pd
import json
from IPython.display import Markdown, display
from pandas.api.types import CategoricalDtype

### IMPORT: Importieren eigenes Packet TreeQinetic

Das Packet TreeQinetic wurde vom Autor (Kyell Jensen) zum einfachen Analysieren, Plotten und zur Interpretation der TXT-Messdaten der Picus TreeQinetic Elastometer und Inclinometer der Firma IML Instrumenta Mechanik Labor Electronic GmbH geschrieben (https://www.iml-electronic.de/produkt/picus-treeqinetic/). Nachfolgend wird das Packet und einige dort definierten Klassen importiert.

In [ ]:
import treeqinetic as ptq
# ptq.help() # Test

Lade allgemeine Export-Funktionen, um die Daten als Latex-Tabellen zu exportieren

In [ ]:
from kj_core.utils.latex_export import (
    save_latex_table,
    build_data_dict_df
)

### IMPORT: Projekt Konfiguration laden

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [ ]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    analyse_name,
    working_directory,
    data_path,
    data_export_directory,
    latex_export_directory
)

## IMPORT: TreeQinetic Daten laden

Aus dem Zugversuchsset wurden 4 Elastometer und 3 Inclinometer verwendet.

Die Elastometer waren auf dem rechten und linken Stämmling auf der Außenseite in zwei Ebenen platziert. Beim Zusammenziehen der Stämmlinge messen diese entsprechnd eine Faserdehnung, beim Ausschwingen der Stämmlinge über ihre Ruhelage hinaus nach außen eine Faserstauchung.

Die Inclinometer Daten werden hier ebefalls geladen, erschienen aber weniger geeignet zur Auswertung und werden entspricht nicht weiter berücksichtigt.

Die Funktion 'ptq.setup' erstellt div. Instanzen, die für das Paket notwendig sind (CONFIG, LOG_MANAGER, PLOT_MANAGER). 

Über die Klasse 'ptq.Series' wird eine neue Messreihe initialisiert und als 'ptq_series' gespeichert. Im Verzeichnis ptq_data_path finden sich die PTQ Daten als TXT von insgesamt 29 Messungen. Eine Datei enthält jeweils die Daten für alle Inclinometer und Elastometer.

## IMPORT: Datendokumentation laden

In [ ]:
ptq.setup(working_directory=working_directory, log_level="info", safe_logs_to_file=True)

ptq_data_path = data_path / 'PTQ/data_txt'
ptq_series = ptq.classes.Series(name=analyse_name, path=ptq_data_path)

# Relevante Elastometer
elasto_names = ["Elasto(90)", "Elasto(92)", "Elasto(95)", "Elasto(98)"]

In [ ]:
ptq_data_dict = ptq_series.create_oscillations_data_dict()
# In DataFrame umwandeln
ptq_data_dict_df= build_data_dict_df(ptq_data_dict)

# In Markdown umwandeln und anzeigen
md_text = ptq_data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

## ANALYSE: Explorative Datenanalyse

Übersicht über alle vom PTQ erfassten Daten über alle Messungen gemeinsam (Elastos und Inclinos).

In [ ]:
ptq_df = ptq_series.get_measurements_df()
ptq_df

In [ ]:
ptq_df.describe()

In [ ]:
ptq_df_elasto_summary = ptq_df[elasto_names].describe()
ptq_df_elasto_summary

### PLOTTING: Daten aller Messungen bzw. Elastometer

Plotten der 4 verwendeten Elastometer in einem Plot für jede Messung. Die Plots werden im Verzeichnis ptq/plots/multi_sensors_vs_time_1/ abgelegt.
Für alle Messungen und Elastometer ist gut zu erkennen, wie die Faserdehnung während des zusammen ziehen der Stämmlinge zunimmt, dann im Moment des Realises plötzlich abfällt, um in Folge harmonisch gedämpft auszuschwingen (nährungsweise).

In [ ]:
ptq_series.plot_measurement_sensors(sensor_names=elasto_names)

## ANALYSE: Bestimmung von Schwingungsparametern

### Selektieren der relevanten Bereiche

Selektiere die Bereiche nach dem Release, bei dem es zu einer harmonisch gedämpften Schwingung kommt. Die Methode Series.get_oscillations sucht im Standardfall nach einem Bereich in den Messdaten mit einer Länge von 20 Sekunden. Der Anfangszeitpunkt wird durch einen plötzlichen Abfall der Dehnung auf unter Null bestimmt, bei dem die Steigung mindestens -25 beträgt. Die Suche nach dem Startzeitpunkt beginnt erst 60 Sekunden nach Messungsbeginn. Der entsprechende Code befindet sich im Paket classes/measurement.py und utils/select_oscillation.py. Die so isolierten Bereiche werden als Instanzen der Klasse Oscillation initialisiert. Weitere Parameter wie Amplitude, Frequenz und Dämpfung werden direkt berechnet.
Parameter:
- sensor_names: Eine Liste der Sensornamen, für die die Schwingungsdaten identifiziert werden sollen.
- min_time_default: Die Mindestzeitspanne nach Beginn der Messung, nach der die Suche nach Schwingungen beginnt (Standard: 60 Sekunden).
- min_value: Der minimale Wertschwellenwert, damit Sensordaten als gültig betrachtet werden.
- threshold_slope: Der Steigungsschwellenwert, um den Beginn einer Schwingung zu bestimmen.
- duration: Die Dauer, für die die Schwingungsdaten extrahiert werden sollen.

In [ ]:
ptq_series.get_oscillations(
    sensor_names=elasto_names,
    min_time_default=60,
    min_value=50,
    threshold_slope=-50,
    duration=17.5
)

### Selektion optisch prüfen in Plots

Plotten der relevanten Sensoren bzw. der selektierten Bereiche. Die Plots werden im Verzeichnis ptq/plots/select_oscillations_single/ bzw. ptq/plots/select_oscillations_combined/ gespeichert. In einem Combined-Plot werden alle 4 Elastometer einer Messung gemeinsam dargestellt. Hier wird manuell anhand der Plots geprüft, ob für alle Messungen und Sensoren der richtige Bereich ausgewählt wurde.

In [ ]:
ptq_series.plot_oscillations_for_measurements(sensor_names=elasto_names, combined=False)
ptq_series.plot_oscillations_for_measurements(sensor_names=elasto_names, combined=True)

### Anpassen der harmonisch gedämpften Schwingung

Aus der PTQ-Messreihe 'ptq_series' wird über `get_oscillations_list` für alle Messungen, getrennt für jeden Sensor (Elastometer), die `Oscillation`-Instanz in eine Liste zusammengeführt. Entsprechend gibt es für jede PTQ-Messung 4 `Oscillation`-Instanzen (für die 4 Elastometer).

`oscillation.fit` passt alle Schwingungsdaten mit einer allgemeinen Funktion für harmonisch gedämpfte Schwingungen an:
\[
y(t) = A \cdot e^{-\delta t} \cdot \cos(\omega_d \cdot t + \phi) + y_0
\]

#### Parameterbeschreibung:
- `A` (Anfangsamplitude): Der Anfangswert der Amplitude der Schwingung. Dieser Parameter bestimmt die initiale Höhe der Schwingungsamplitude.
- `δ` (Dämpfungskoeffizient): Dieser Wert bestimmt, wie schnell die Amplitude der Schwingung mit der Zeit abnimmt. Ein höherer Wert führt zu einer schnelleren Dämpfung der Schwingung.
- `ω_d` (gedämpfte Kreisfrequenz): Die Frequenz der gedämpften Schwingung in Radiant pro Sekunde. Dieser Parameter bestimmt, wie schnell die Schwingung oszilliert.
- `φ` (Phasenwinkel): Der Anfangsphasenwinkel der Schwingung. Dieser Wert bestimmt den Startpunkt der Schwingung im Schwingungszyklus.
- `y_0` (Vertikale Verschiebung): Dieser Parameter verschiebt die gesamte Schwingungskurve vertikal und ermöglicht es, die Schwingung an die mittlere Position der Daten anzupassen.
- `t_0` (Horizontale Verschiebung): Dieser Parameter verschiebt die gesamte Schwingungskurve horizontal über die Zeit und ermöglicht es, die Schwingung an den spezifischen Startpunkt der gemessenen Schwingung anzupassen.

(siehe `ptq/analyse/fitting_function.py`)

#### Zusätzliche Parameter und Konfigurationen:
- **Startwerte und Grenzwerte:** Für die Optimierung der Parameter in `scipy.curve_fit` werden Startwerte und Grenzwerte für jeden Parameter übergeben (in `ptq/config.py` definiert).
- **Qualitätsmetriken:** Zur Bewertung der Anpassungsgüte werden Metriken wie MAE (mittlerer absoluter Fehler), RMSE (Root Mean Square Error), und \( R^2 \) (Bestimmtheitsmaß) berechnet. Zusätzlich werden normalisierte Varianten (NRMSE und NMAE) zur besseren Vergleichbarkeit verwendet.
- **Warnungen bei Überschreitung der Grenzwerte:** Wenn die für eine Metrik definierten Grenzwerte überschritten werden, wird eine Warnung im Log-Protokoll vermerkt, um auf mögliche Probleme bei der Anpassung hinzuweisen (in `ptq/config.py` definiert). Auf Basis dieser Warnung können:
  - Start- und Grenzwerte sowie die Methodik angepasst werden.
  - Betroffene Datensätze später ausgeschlossen werden, um fehlerhafte Anpassungen zu vermeiden.
- **Interpolation:** Diese Option aktiviert die Interpolation der Datenpunkte, um eine ausreichende Dichte für `curve_fit` zu gewährleisten. Hierbei wird `scipy.interpolate.PchipInterpolator` verwendet, um Über- und Unterschwingungen, die nicht in den Originaldaten vorhanden sind, zu vermeiden. Nach optischer Prüfung zeigte diese Methode die besten Ergebnisse.

#### Visualisierungsoptionen:
- **Plot:** Wenn auf `True` gesetzt, wird für jede Oscillation ein Plot der angepassten Funktion zusammen mit den Originaldaten erstellt und in `working_dir/PTQ/plots/` gespeichert.
- **Plot-Fehlerverteilung:** Wenn `plot_error` auf `True` gesetzt ist, wird ein Histogramm der Fehlerverteilung (Residuen) für jeden Fit erstellt und ebenfalls in `working_dir/PTQ/plots/` gespeichert.

In [ ]:
ptq_oscillations_ls = ptq_series.get_oscillations_list()

initial_param = {
    "initial_amplitude": 170,
    "damping_coeff": 0.32,
    "frequency_damped": 0.44,
    "phase_angle": 0,
    "y_shift": 0,
    "x_shift": 0
}

param_bounds = {
    "initial_amplitude": (150, 250),
    "damping_coeff": (0.1, 1),
    "frequency_damped": (0.35, 0.58),
    "phase_angle": (-0.2, 0.2),
    "y_shift": (-60, 60),
    "x_shift": (-0.25, 0.75),
}

metrics_warning = {
    "pearson_r": (0.75, 1),
    "nrmse": (0, np.inf),
    "mae": (0, np.inf),
    "nmae": (0, 0.10)
}

for oscillation in ptq_oscillations_ls:
    oscillation.fit(
        initial_param,
        param_bounds,
        optimize_criterion="mae",
        metrics_warning=metrics_warning,
        plot=False,
        plot_error=False,
        dir_add="",
        interpolate=True
    )

### Fehlerverteilung der Funktionsanpassung an Messdaten

Die Funktion sammelt für alle Oscillation-Objekte die Fehler-Arrays der Anpassung und normalisiert die Fehler (um Unterschiede in der Skalierung zu entfernen).
Anschließend werden die Fehler für alle Messungen A) für alle Sensoren gemeinsam und B) getrennt für jeden Sensor geplotet. Es werden Q-Q-Plot, Violin-Plot und Histogramme für den gleichen Sachverhalt erstellt und in `working_directory\PTQ\plots\series_osc_errors` abgelegt.

- `trim_hist_percent`: Beschneidet die Daten Links und Rechts um die äußersten x Prozent, da die Verteilung im Zentrum sonst kaum zu bewerten ist. Wirkt sich nur auf die Histogramme aus.

In [ ]:
all_normalized_errors = ptq_series.plot_osc_errors(plot_qq=True, plot_violin=True, plot_hist=True, hist_trim_percent=5)

## ANALYSE: Metadaten bzw. Zusammenfassung aller Dehnungs- und Schwingungsdaten

Bewerte die Güte der Anpassung

In [ ]:
osc_optimization_details_df = ptq_series.get_osc_optimization_details_df()
osc_optimization_details_df.describe()

'ptq_series.get_oscillations_df' fasst aus allen Oscillation-Instanzen der Messreihe ('ptq_series') die Schwingungsparameter als pandas.DataFrame zusammen

In [ ]:
ptq_metadata_df = ptq_series.get_oscillations_df()

### ANALYSE: Bewertung der Anpassungsgüte

In [ ]:
no_metrics_warning_df = osc_optimization_details_df[ptq_metadata_df['metrics_warning']==False]

In [ ]:
#no_metrics_warning_df.describe()

In [ ]:
metrics_warning_df = osc_optimization_details_df[ptq_metadata_df['metrics_warning']==True]

In [ ]:
#metrics_warning_df.describe()

#### LATEX EXPORT: Exportiere Statistiken der Anpassungsgüte reduziert als Latex-Tabelle

In [ ]:
select_cols = ['pearson_r', 'mae', 'nmae', 'rmse', 'nrmse']
describe_agg = ['mean', 'std', 'min', 'max']
metrics_warning_df_list = [
    ("Alle", osc_optimization_details_df),
    ("Warnung", metrics_warning_df),
    ("Keine", no_metrics_warning_df)
]

# Erstelle eine Liste, um die statistischen DataFrames mit einer zusätzlichen Spalte "Gruppe" zu sammeln
stats_list = []

for label, df in metrics_warning_df_list:
    df_stats = df[select_cols].describe().loc[describe_agg]
    df_stats['Gruppe'] = label  # Füge die Gruppenbezeichnung als Spalte hinzu
    # Setze "Gruppe" als ersten Index, falls du dies bevorzugst
    df_stats = df_stats.set_index('Gruppe', append=True)
    # Ersetze "50%" im MultiIndex der Zeilen durch "median" (auf Ebene 1)
    df_stats = df_stats.swaplevel(0, 1)
    stats_list.append(df_stats)

# Verbinde die DataFrames entlang der Spaltenachse
combined_stats = pd.concat(stats_list, axis=0)
# Ersetze Spaltennamen durch Formelzeichen
combined_stats.columns = [ptq_data_dict[col]["Zeichen"] for col in select_cols]
combined_stats

In [ ]:
# Exportiere die kombinierte Tabelle als LaTeX
latex_string_combined = combined_stats.to_latex(
    index=True,
    escape=False,
    float_format="{:0.3f}".format,
    column_format="ll|r|rr|rr"
)
caption = "Feldversuch 2 - Ergebnisse, Schwingung, Anpassungsgüte"
caption_long = f"Feldversuch 2 - Ergebnisse, Bewertung der Anpassung der harmonisch gedämpften Schwingung an die Daten, Gruppe 'Alle' = Alle 116 Datensätze, 'Warnung' = 9 Datensätze mit r < 0,75, 'Keine' =  107 Datensätze mit r >= 0,75"


save_latex_table(latex_string_combined, caption, latex_export_directory, 
                   caption_long)

## EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [ ]:
# DataFrame als Feather
ptq_metadata_df.to_feather(data_export_directory / "ptq.feather")

# Dict als JSON (UTF-8, sauber eingerückt)
with open(data_export_directory / "ptq_data_dict.json", "w", encoding="utf-8") as f:
    json.dump(ptq_data_dict, f,  indent=4, ensure_ascii=False)
    
ptq_metadata_df.to_csv(data_export_directory / "ptq.csv", sep=";", index=False, encoding="utf-8")

## LATEX-EXPORT: Datendokumentation als Latex-Tabelle exportieren (.tex)

In [ ]:
ptq_data_dict_df = build_data_dict_df(ptq_data_dict, escape_index=True, select_latex_fields=True)

latex_string = ptq_data_dict_df.to_latex(index=False, escape=False)
caption = "Feldversuch 2 - PTQ Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)